In [2]:
import pandas as pd

# Load the dataset
df = pd.read_excel('Online retail.xlsx')
print(df.head())
print(df.info())  # Check data types and null values


  shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
0                             burgers,meatballs,eggs                                                                                                                                                                             
1                                            chutney                                                                                                                                                                             
2                                     turkey,avocado                                                                                                                                                                             
3  mineral water,milk,energy bar,whole wheat rice...                                            

In [3]:
# Drop rows with any missing values
df.dropna(inplace=True)


In [4]:
df.drop_duplicates(inplace=True)


In [9]:
import pandas as pd

# Load the dataset
df = pd.read_excel('Online retail.xlsx', header=None)  # Load without headers if the column is unnamed

# Rename the column to 'Items'
df.columns = ['Items']

# Split the items in each row
df['Items'] = df['Items'].str.split(',')

# Convert each transaction (row) into a list of items
transactions = df['Items'].tolist()


In [10]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Encode the transactions
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
basket = pd.DataFrame(te_ary, columns=te.columns_)

# Run the Apriori algorithm to find frequent itemsets
frequent_itemsets = apriori(basket, min_support=0.02, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
print(rules.head())


       antecedents     consequents  antecedent support  consequent support  \
0        (burgers)          (eggs)            0.087188            0.179709   
1           (eggs)       (burgers)            0.179709            0.087188   
2   (french fries)       (burgers)            0.170911            0.087188   
3        (burgers)  (french fries)            0.087188            0.170911   
4  (mineral water)       (burgers)            0.238368            0.087188   

    support  confidence      lift  leverage  conviction  zhangs_metric  
0  0.028796    0.330275  1.837830  0.013128    1.224818       0.499424  
1  0.028796    0.160237  1.837830  0.013128    1.086988       0.555754  
2  0.021997    0.128705  1.476173  0.007096    1.047650       0.389069  
3  0.021997    0.252294  1.476173  0.007096    1.108844       0.353384  
4  0.024397    0.102349  1.173883  0.003614    1.016889       0.194486  


In [11]:
from mlxtend.frequent_patterns import apriori, association_rules

# Set a minimum support threshold and find frequent itemsets
frequent_itemsets = apriori(basket, min_support=0.02, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
print(rules.head())


       antecedents     consequents  antecedent support  consequent support  \
0        (burgers)          (eggs)            0.087188            0.179709   
1           (eggs)       (burgers)            0.179709            0.087188   
2   (french fries)       (burgers)            0.170911            0.087188   
3        (burgers)  (french fries)            0.087188            0.170911   
4  (mineral water)       (burgers)            0.238368            0.087188   

    support  confidence      lift  leverage  conviction  zhangs_metric  
0  0.028796    0.330275  1.837830  0.013128    1.224818       0.499424  
1  0.028796    0.160237  1.837830  0.013128    1.086988       0.555754  
2  0.021997    0.128705  1.476173  0.007096    1.047650       0.389069  
3  0.021997    0.252294  1.476173  0.007096    1.108844       0.353384  
4  0.024397    0.102349  1.173883  0.003614    1.016889       0.194486  


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
# Filter rules with a minimum support, confidence, and lift
meaningful_rules = rules[(rules['support'] >= 0.02) &
                         (rules['confidence'] >= 0.5) &
                         (rules['lift'] >= 1.2)]
print(meaningful_rules)


Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction, zhangs_metric]
Index: []


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1. **Analyzing Generated Rules**

  The rules reveal interesting relationships among frequently purchased items. I focused on key metrics like support, confidence, and lift to identify significant patterns:

 * Lift: Rules with high lift values, such as (burgers → eggs) with a lift of 1.84, show that these items are bought together more frequently than expected by random chance. This suggests that customers who buy burgers are likely to buy eggs as well, which is valuable information for creating bundled offers.

 * Support and Confidence:

    * For (burgers → eggs), the support of 0.0288 tells me that this combination appears in roughly 2.88% of transactions, and the confidence of 33.03% indicates that when customers buy burgers, there’s a 33% chance they’ll also purchase eggs. This high confidence level highlights a strong complementary relationship that could be leveraged for cross-selling.
2. **Interpreting Insights**

 These patterns offer valuable insights into customer purchasing behavior:

 * Frequent Itemsets:

    * Mineral Water and Burgers: The rule (mineral water → burgers) with a lift of 1.17 and support of 2.44% implies that these items are often bought together, hinting that customers may be buying them for meals or snacks. I could use this insight to suggest these items together in displays or online shopping recommendations.
    * French Fries and Burgers: The strong association between burgers and french fries, with a lift of 1.48, confirms their popularity as a meal combination, which could benefit from promotional bundling or in-store pairing.
 * Confidence Levels:

    * The high-confidence rule (burgers → french fries), with a confidence of 25.23%, suggests that when customers buy burgers, there’s a significant chance they’ll also buy french fries. This pattern aligns with common meal combinations, making it a good target for cross-promotions.
 * Lift:

    * Rules with lift values over 1.0, like (burgers → eggs) and (eggs → burgers), highlight strong associations that go beyond random chance. These pairings could be ideal for combo offers or strategic shelf placements to enhance sales.

# Interview Questions

**1. What is Lift and Why is it Important in Association Rules?**

 Ans. **Lift** measures the strength of an association rule relative to random chance. It is calculated as the ratio of the observed co-occurrence of items to their expected co-occurrence if they were independent.

  **Importance:** Lift values greater than 1 indicate a strong association between items, meaning they’re more likely to be purchased together than by random chance. It helps in identifying meaningful product relationships.

**2. What is Support and Confidence, and How Do You Calculate Them?**

 Ans. **Support** is the proportion of transactions that contain a particular item or itemset, indicating how frequently an item appears in the dataset.

 **Confidence** is the likelihood of an item Y being purchased when item X is bought.

 **Use in Association Rules:** Both metrics help in determining the strength of the relationship between items and filtering out insignificant rules.

**3. What are Some Limitations or Challenges of Association Rules Mining?**

 Ans. **Scalability:** Large datasets with many items can lead to a high computational load, making association rule mining resource-intensive.

  **Interpretability:** With too many rules, it’s challenging to interpret or identify meaningful patterns without prior filtering.

  **Sparsity of Data:** Many transactions may contain unique item combinations, leading to fewer common patterns.

  **Choice of Thresholds:** Setting support, confidence, and lift thresholds can be challenging; thresholds that are too high might miss important associations, while those that are too low may yield insignificant rules.
